In [ ]:
import os
import site

try:
    site_packages = site.getsitepackages()[0]
    nvidia_path = os.path.join(site_packages, 'nvidia')
    
    cudnn_path = os.path.join(nvidia_path, 'cudnn', 'lib')
    cuda_path = os.path.join(nvidia_path, 'cuda_runtime', 'lib')
    
    old_ld = os.environ.get('LD_LIBRARY_PATH', '')
    os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{cuda_path}:{old_ld}"
    
    os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'
    
    print("NVIDIA Library paths arranged successfully")
    
except Exception as e:
    print(f"Path warning: {e}")


import tensorflow as tf

print(f"TensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs: {gpus}")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import classification_report

# GPU Bellek Ayarı (Memory Growth)
# Bu kısım importlardan hemen sonra çalışmalı
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Detected and memory setting done: {gpus}")
    except RuntimeError as e:
        print(e)

✅ NVIDIA Kütüphane yolları başarıyla ayarlandı.


2025-12-15 14:35:01.795047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-15 14:35:01.795254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-15 14:35:01.884560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-15 14:35:02.062148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 14:35:03.564588: W tensorflow/compiler/tf2

TensorFlow Sürümü: 2.15.0
Algılanan GPU'lar: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-12-15 14:35:05.360285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 14:35:05.801119: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 14:35:05.801971: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

GPU Algılandı ve Bellek Ayarı Yapıldı: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import pandas as pd
import numpy as np

In [55]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [81]:
def check_df(dataframe):
    """
    Checks the overall structure and key metrics of a DataFrame.

    Args:
        dataframe (pd.DataFrame): DataFrame to inspect.

    Returns:
        None: Prints shape, data types, head, tail, missing values, and quantiles.
    """
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(5))
    print("##################### Tail #####################")
    print(dataframe.tail(5))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print('##################### Unique Values #####################')
    print(dataframe.nunique())
    print("##################### Duplicates #####################")
    print(dataframe.duplicated().sum())
    print("##################### Quantiles #####################")
    # Uncomment below to include quantile information
    #print(dataframe[[col for col in dataframe.columns if dataframe[col].dtypes != "O"]].quantile([0, 0.05, 0.50, 0.75, 0.95, 0.99, 1]).T)
    print(dataframe.describe().T)


In [ ]:
df_train = pd.read_csv("data/train.csv", encoding="UTF-8", engine="python", encoding_errors="replace")#replaces damaged bytes with "\ufffd"

In [82]:
check_df(df_train)

##################### Shape #####################
(440679, 2)
##################### Types #####################
text     object
label    object
dtype: object
##################### Head #####################
                                                text     label
0  ürünü hepsiburadadan alalı 3 hafta oldu. orjin...  Positive
1  ürünlerden çok memnunum, kesinlikle herkese ta...  Positive
2      hızlı kargo, temiz alışveriş.teşekkür ederim.  Positive
3               Çünkü aranan tapınak bu bölgededir .      Notr
4  bu telefonu başlıca alma nedenlerim ise elimde...  Positive
##################### Tail #####################
                                                     text     label
440674  Ayrıca burç yorumları ve çapraz bulmaca da der...      Notr
440675  günümüz de ssd olmazsa olmaz bir donanım artık...  Positive
440676  kullandım ve çok memnun kaldım. ocak başında d...  Positive
440677                Adını Lenkeran şehrinden almıştır .      Notr
440678  Bu dergilerde sosy

In [71]:
df_test = pd.read_csv("data/test.csv", encoding="UTF-8", engine="python", encoding_errors="replace")

In [83]:
check_df(df_test)

##################### Shape #####################
(48965, 2)
##################### Types #####################
text     object
label    object
dtype: object
##################### Head #####################
                                                text     label
0      Kral akbaba dikkat çekici renklere sahiptir .      Notr
1   ısrarla korkutmayı başarıyor. sanki korku çok...  Positive
2  Neşe ve Üzüntü köprünün kırılmaya başlamasıyla...      Notr
3  i phone 5 ten sonra gene 4'' ekranı tercih ett...  Positive
4    Beşinci sezonda diziye yeni oyuncular katıldı .      Notr
##################### Tail #####################
                                                    text     label
48960  Fransa bayrağı diğer kırmızı zeminden beyaz bi...      Notr
48961  Yine aynı yıl türkü dalında Murat Çobanoğlu il...      Notr
48962                           Kurgunu skiyim oç evladı  Negative
48963  Şarkı daha sonrasında Damian Marley tarafından...      Notr
48964  berrak bir ürün ancak ken

In [ ]:
#damaged rows filtering, these can be considered to be dropped
damaged_rows_train = df_train[df_train["text"].str.contains("\ufffd", na=False)]
damaged_rows_test = df_test[df_test["text"].str.contains("\ufffd", na=False)]

print(f"Total damaged rows in train: {len(damaged_rows_train)}")

print(damaged_rows_train.head())

print(f"Total damaged rows in test: {len(damaged_rows_test)}")

print(damaged_rows_test.head())

Total damaged rows in train: 7
                                                     text     label
31512   - su akıtmıyor: adamlar kullanam klavuzuna yaz...  Positive
55634   -kargocu arkadaşlar ürünü bir bayan olarak taş...  Positive
64093   - kullanım tarifindeki 'hazneye sıcak su koyun...  Positive
102817  -kamerasına laf edilmiş. çıktığı dönemin en iy...  Positive
332479  - karşı taraf sesimden çok memnun ama ben karş...  Positive
Total damaged rows in test: 0
Empty DataFrame
Columns: [text, label]
Index: []


**"kullanam klavuzu", encoding="hırt"**

**TASK**

4 different models ([TF-IDF with Multinomial Naive Bayes and Binary Naive Bayes] + [ANN with Word2Vec and FastText]) will be trained and compared.

**ROADMAP**

Preprocessing steps will be applied on data according to models they will be fed to.

***For Bayesian Model:***
- Lowecase transformation
- Special characters cleaning (Punctuations etc.)